In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
print(spark)

In [0]:
sc.stop()

In [0]:
from pyspark import SparkContext, SparkConf

# New Spark SQL Context
from pyspark.sql import SQLContext
conf = SparkConf().setAppName("Spark_ProjLogAnalysis").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)
sqlcontext = SQLContext(sc)

/databricks/spark/python/pyspark/sql/context.py:82: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [0]:
print(spark)

In [0]:
logs_DF = spark.read.csv("dbfs:/FileStore/tables/all_transaction_1.csv", header=True, inferSchema=True)
logs_DF.show(10,truncate=False)

+------------------------------------+---------------------------------------------------------------------------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|ProductCode                         |Description                                                                            |Brand       |Category|SubCategory        |ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|InvoiceDate        |Mode          |InvoiceNumber|CustomerId  |Country  |
+------------------------------------+---------------------------------------------------------------------------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|e6fa885e-5f14-4e8c-8651-0c4e1c12be7b|Landau Pants 2

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import date_format,year,col,to_timestamp
logs_DF1=logs_DF.withColumn("Year",year(logs_DF.InvoiceDate)).withColumn("Month",month(logs_DF.InvoiceDate)).withColumn("Quarter",quarter(logs_DF.InvoiceDate))

In [0]:
logs_DF1.createOrReplaceTempView("sales_table")

####a. Find the top five performing mobile brands for the months of November, December
####and January r each year.

In [0]:
spark.sql("""select sales_year,sales_month,brand,highest_selling from (
select brand,year as sales_year,month as sales_month,row_number() over(partition by year,month order by sum(quantity) desc) as rank,sum(quantity) as highest_selling from sales_table where SubCategory='mobile' and month in (12,11,1)  group by brand,sales_year,sales_month)a where rank<=5 order by sales_year,sales_month,highest_selling desc""").show()

+----------+-----------+--------+---------------+
|sales_year|sales_month|   brand|highest_selling|
+----------+-----------+--------+---------------+
|      2015|          1| samsung|            377|
|      2015|          1|   apple|            223|
|      2015|          1|   nokia|            168|
|      2015|          1|     blu|            148|
|      2015|          1|motorola|            127|
|      2015|         11| samsung|            448|
|      2015|         11|     blu|            234|
|      2015|         11|   apple|            200|
|      2015|         11|   nokia|            185|
|      2015|         11|      lg|            165|
|      2015|         12| samsung|            517|
|      2015|         12|     blu|            249|
|      2015|         12|   nokia|            212|
|      2015|         12|   apple|            184|
|      2015|         12|     htc|            158|
|      2016|          1| samsung|            361|
|      2016|          1|   apple|            216|


####b. Find the top five selling products for the months of November, December and
####January for each year.

In [0]:
spark.sql("""select sales_year,sales_month,subcategory,highest_selling from (
select subcategory,year as sales_year,month as sales_month,row_number() over(partition by year,month order by sum(quantity) desc) as rank,sum(quantity) as highest_selling from sales_table where month in (12,11,1)  group by subcategory,sales_year,sales_month)a where rank<=5 order by sales_year,sales_month,highest_selling desc""").show()

+----------+-----------+-----------------+---------------+
|sales_year|sales_month|      subcategory|highest_selling|
+----------+-----------+-----------------+---------------+
|      2015|          1|all women's shoes|           2891|
|      2015|          1|           mobile|           2255|
|      2015|          1|      men's shoes|           1835|
|      2015|          1|  shoes & jewelry|            955|
|      2015|          1|    women's shoes|            829|
|      2015|         11|           mobile|           2701|
|      2015|         11|      men's shoes|           2186|
|      2015|         11|all women's shoes|           2044|
|      2015|         11|  shoes & jewelry|            870|
|      2015|         11|    women's shoes|            586|
|      2015|         12|all women's shoes|           2970|
|      2015|         12|           mobile|           2935|
|      2015|         12|      men's shoes|           2010|
|      2015|         12|  shoes & jewelry|            91

####a. Number of products sold along with revenue aggregated quarterly

In [0]:
spark.sql("SELECT year,quarter,category,sum(quantity) as no_products_sold,sum(transactionamount) as quarter_revenue from sales_table group by year,quarter,category order by year,quarter,no_products_sold desc").show(truncate=False)

+----+-------+-------------+----------------+-------------------+
|year|quarter|category     |no_products_sold|quarter_revenue    |
+----+-------+-------------+----------------+-------------------+
|2015|1      |clothing     |32683           |1.89308525E8       |
|2015|1      |electronics  |7872            |1.10534375E8       |
|2015|1      |shoes        |2134            |8619415.0          |
|2015|1      |apple carplay|4               |86240.0            |
|2015|2      |clothing     |32481           |1.87920992E8       |
|2015|2      |electronics  |7434            |2.23020102342676E14|
|2015|2      |shoes        |2154            |1.2181617E7        |
|2015|3      |clothing     |33517           |2.08341854E8       |
|2015|3      |electronics  |7497            |9.8171612E7        |
|2015|3      |shoes        |2210            |1.106511E7         |
|2015|4      |clothing     |33537           |1.80563516E8       |
|2015|4      |electronics  |9917            |1.2996053E8        |
|2015|4   

####b. Number of products sold in each category aggregated quarterly

In [0]:
spark.sql("SELECT year,quarter,subcategory,sum(quantity) as no_products_sold,sum(transactionamount) as quarter_revenue from sales_table group by year,quarter,subcategory order by year,quarter,no_products_sold desc").show(truncate=False)

+----+-------+----------------------------------+----------------+---------------+
|year|quarter|subcategory                       |no_products_sold|quarter_revenue|
+----+-------+----------------------------------+----------------+---------------+
|2015|1      |all women's shoes                 |7155            |4.8507606E7    |
|2015|1      |mobile                            |6487            |7.5902435E7    |
|2015|1      |men's shoes                       |5854            |3.1296321E7    |
|2015|1      |shoes & jewelry                   |2587            |1.3536425E7    |
|2015|1      |women's shoes                     |2134            |9018086.0      |
|2015|1      |all men's shoes                   |1338            |7802228.0      |
|2015|1      |athletic                          |623             |4346706.0      |
|2015|1      |women's accessories               |619             |3154683.0      |
|2015|1      |sunglasses & eyewear accessories  |592             |4371129.0      |
|201

####c. A number of products sold for each brand aggregated quarterly.

In [0]:
spark.sql("SELECT year,quarter,brand,sum(quantity) as no_products_sold,sum(transactionamount) as quarter_revenue from sales_table group by year,quarter,brand order by year,quarter,no_products_sold desc").show(truncate=False)

+----+-------+----------+----------------+---------------+
|year|quarter|brand     |no_products_sold|quarter_revenue|
+----+-------+----------+----------------+---------------+
|2015|1      |nike      |2642            |1.4348817E7    |
|2015|1      |samsung   |1174            |2.3020928E7    |
|2015|1      |apple     |646             |1.2887868E7    |
|2015|1      |new       |634             |2765798.0      |
|2015|1      |puma      |618             |3553963.0      |
|2015|1      |reebok    |539             |3039337.0      |
|2015|1      |blu       |533             |5154877.0      |
|2015|1      |vans      |457             |3558485.0      |
|2015|1      |nokia     |441             |4170432.0      |
|2015|1      |yu&yu     |384             |1983051.0      |
|2015|1      |sony      |382             |7414715.0      |
|2015|1      |skechers  |380             |1634087.0      |
|2015|1      |htc       |348             |4018371.0      |
|2015|1      |adidas    |348             |1557710.0     

####a. Top 5 profit margin generating products for a given month.

In [0]:
spark.sql("""select sales_year,sales_month,subcategory,ProfitMargin from (
select subcategory,year as sales_year,month as sales_month,row_number() over(partition by year,month order by sum(case when Marginpercentage-Discountpercentage > 0 then 1 else 0 end) desc) as rank,sum(case when Marginpercentage-Discountpercentage > 0 then 1 else 0 end) as ProfitMargin from sales_table group by subcategory,sales_year,sales_month)a where rank<=5 order by sales_year,sales_month,ProfitMargin desc""").show()

+----------+-----------+-----------------+------------+
|sales_year|sales_month|      subcategory|ProfitMargin|
+----------+-----------+-----------------+------------+
|      2015|          1|all women's shoes|         328|
|      2015|          1|           mobile|         260|
|      2015|          1|      men's shoes|         225|
|      2015|          1|  shoes & jewelry|         136|
|      2015|          1|    women's shoes|         116|
|      2015|          2|           mobile|         239|
|      2015|          2|all women's shoes|         215|
|      2015|          2|      men's shoes|         178|
|      2015|          2|    women's shoes|          82|
|      2015|          2|  shoes & jewelry|          76|
|      2015|          3|      men's shoes|         254|
|      2015|          3|all women's shoes|         251|
|      2015|          3|           mobile|         231|
|      2015|          3|  shoes & jewelry|         110|
|      2015|          3|    women's shoes|      

####4. Brands that need marketing
####a. Brand wise segregation of best performing categories profits wise

In [0]:
spark.sql("""select brand,SubCategory,products_sold,ProfitPercentage,total_revenue,(total_revenue / ProfitPercentage) as net_profit from (select brand,SubCategory,(Marginpercentage-Discountpercentage) as ProfitPercentage,sum(Quantity) as products_sold ,sum(transactionamount) as total_revenue from sales_table group by brand,ProfitPercentage,SubCategory) where ProfitPercentage>0 group by brand,SubCategory,products_sold,total_revenue,ProfitPercentage,net_profit order by net_profit desc""").show(truncate=False)

+----------+---------------------------+-------------+----------------+--------------+--------------+
|brand     |SubCategory                |products_sold|ProfitPercentage|total_revenue |net_profit    |
+----------+---------------------------+-------------+----------------+--------------+--------------+
|pioneer   |parts & accessories        |56           |10              |3.122280028E15|3.122280028E14|
|samsung   |mobile                     |2980         |10              |4.1856129E7   |4185612.9     |
|annie     |women's shoes              |12           |20              |7.88886E7     |3944430.0     |
|apple     |mobile                     |1323         |10              |2.5835397E7   |2583539.7     |
|blu       |mobile                     |1524         |10              |2.0141338E7   |2014133.8     |
|nike      |men's shoes                |2685         |10              |1.6590182E7   |1659018.2     |
|sony      |mobile                     |802          |10              |1.2607938E7

####b. Brand wise segregation of least performing categories orders wise

In [0]:
spark.sql("select brand,SubCategory,sum(Quantity) as products_sold from sales_table group by brand,SubCategory order by products_sold").show(truncate=False)

+---------+----------------------------+-------------+
|brand    |SubCategory                 |products_sold|
+---------+----------------------------+-------------+
|hailey   |women's shoes               |1            |
|liz      |shoes & handbags            |1            |
|cooler   |auto & tires                |1            |
|converse |other wholesale women's accs|1            |
|platinum |jewelry                     |1            |
|luxury   |women's clothing            |1            |
|nwt      |men's work shoes & boots    |1            |
|basic    |shoes & jewelry             |1            |
|capezio  |waist packs                 |1            |
|silvertoe|shoes & jewelry             |1            |
|patrick  |men's clothing              |1            |
|totes    |women's dress shoes         |1            |
|tommy    |intimates & sleepwear       |1            |
|parisa   |all women's shoes           |1            |
|luckers  |all women's shoes           |1            |
|deer     

#### 5. Find the products that should be part of the lighting deals.
#### a. Find the top five selling products in a given month

In [0]:
spark.sql("""select sales_year,sales_month,subcategory,highest_selling from (
select subcategory,year as sales_year,month as sales_month,row_number() over(partition by year,month order by sum(quantity) desc) as rank,sum(quantity) as highest_selling from sales_table group by subcategory,sales_year,sales_month)a where rank<=5 order by sales_year,sales_month,highest_selling desc""").show()

+----------+-----------+-----------------+---------------+
|sales_year|sales_month|      subcategory|highest_selling|
+----------+-----------+-----------------+---------------+
|      2015|          1|all women's shoes|           2891|
|      2015|          1|           mobile|           2255|
|      2015|          1|      men's shoes|           1835|
|      2015|          1|  shoes & jewelry|            955|
|      2015|          1|    women's shoes|            829|
|      2015|          2|           mobile|           2131|
|      2015|          2|all women's shoes|           1993|
|      2015|          2|      men's shoes|           1729|
|      2015|          2|  shoes & jewelry|            646|
|      2015|          2|    women's shoes|            592|
|      2015|          3|      men's shoes|           2290|
|      2015|          3|all women's shoes|           2271|
|      2015|          3|           mobile|           2101|
|      2015|          3|  shoes & jewelry|            98

####b. Find the top five products having the highest margin in a given month

In [0]:
spark.sql("""select sales_year,sales_month,subcategory,highest_margin from (
select subcategory,year as sales_year,month as sales_month,row_number() over(partition by year,month order by sum(Marginpercentage) desc) as rank,sum(Marginpercentage) as highest_margin from sales_table group by subcategory,sales_year,sales_month)a where rank<=5 order by sales_year,sales_month,highest_margin desc""").show()

+----------+-----------+-----------------+--------------+
|sales_year|sales_month|      subcategory|highest_margin|
+----------+-----------+-----------------+--------------+
|      2015|          1|all women's shoes|         16910|
|      2015|          1|           mobile|         13210|
|      2015|          1|      men's shoes|         10940|
|      2015|          1|  shoes & jewelry|          5690|
|      2015|          1|    women's shoes|          5400|
|      2015|          2|           mobile|         11980|
|      2015|          2|all women's shoes|         11160|
|      2015|          2|      men's shoes|          9200|
|      2015|          2|  shoes & jewelry|          3690|
|      2015|          2|    women's shoes|          3530|
|      2015|          3|      men's shoes|         13270|
|      2015|          3|all women's shoes|         13140|
|      2015|          3|           mobile|         12260|
|      2015|          3|  shoes & jewelry|          5440|
|      2015|  

####c. Find the five lowest selling products in a given month.

In [0]:
spark.sql("""select sales_year,sales_month,subcategory,lowest_selling from (
select subcategory,year as sales_year,month as sales_month,row_number() over(partition by year,month order by sum(quantity)) as rank,sum(quantity) as lowest_selling from sales_table group by subcategory,sales_year,sales_month)a where rank<=5 order by sales_year,sales_month,lowest_selling""").show(truncate=False)

+----------+-----------+----------------------------------------------------------+--------------+
|sales_year|sales_month|subcategory                                               |lowest_selling|
+----------+-----------+----------------------------------------------------------+--------------+
|2015      |1          |women's running shoes                                     |1             |
|2015      |1          |link                                                      |1             |
|2015      |1          |wristwatch bands                                          |1             |
|2015      |1          |scuff                                                     |1             |
|2015      |1          |masks & peels                                             |1             |
|2015      |2          |see more saxx 9699 mens vibe orange striped modern fit ...|1             |
|2015      |2          |neutral                                                   |1             |
|2015     